In [58]:
import mediapipe as mp
import cv2
import os
import matplotlib.pyplot as plt
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np
from mediapipe.framework.formats import landmark_pb2
from mediapipe import solutions
import tensorflow

In [59]:
BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

In [60]:
options = HandLandmarkerOptions(
    base_options = BaseOptions(model_asset_path="hand_landmarker.task"),
    running_mode = VisionRunningMode.IMAGE
)

In [61]:
detector = vision.HandLandmarker.create_from_options(options)

In [62]:
#takes in a detector object's hand_landmarks list (detecting the hand as a whole in the image)
#takes in the single_images numpyview off of mp create image

def draw_landmarks_on_hand(image, hand_landmarks_list):
    modified_image = np.copy(image)
    
    if len(hand_landmarks_list) > 0:
        points = []
        for hand_point in hand_landmarks_list[0]:
            hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()

            hand_landmarks_proto.landmark.extend([
                landmark_pb2.NormalizedLandmark(x=hand_point.x, y=hand_point.y, z=hand_point.z) for hand_point in hand_landmarks_list[0]
            ])
            
            points.append([hand_point.x, hand_point.y, hand_point.z])

            solutions.drawing_utils.draw_landmarks(
                modified_image,
                hand_landmarks_proto,
                solutions.hands.HAND_CONNECTIONS,
                solutions.drawing_styles.get_default_hand_landmarks_style(),
                solutions.drawing_styles.get_default_hand_connections_style()
            )
        return True, modified_image, np.array(points)
    else:
        #no hand detected
        return False, None, None

In [63]:
# single_image = mp.Image.create_from_file(single_image)

In [64]:
model = tensorflow.keras.models.load_model("sign_language_model.keras")

In [65]:
label_dictionary = {0:"Hello!", 1: "I Love You!", 2: "Yes!"} #

In [66]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if ret:
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        detection_results = detector.detect(mp_image)
        success, modified_frame, points = draw_landmarks_on_hand(frame, detection_results.hand_landmarks) #pass in only one hand detected

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

        if success:
            points = points.reshape(1, 21, 3)
            prediction = model.predict(points)
            cv2.putText(modified_frame, label_dictionary[np.argmax(prediction, axis=1)[0]], (250, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.25, (0, 0, 255), 2, cv2.LINE_AA)
            cv2.imshow("sign detection", modified_frame)
        else:
            cv2.putText(frame, "No hand detected yet!", (25, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2, cv2.LINE_AA)
            cv2.imshow("sign detection", frame)        
cap.release()
cv2.destroyAllWindows()